In [2]:
import processing
import ds
import ml
import model_saving
import TS
import TA
# Load the necessary packages and modules
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import numpy as np
import math as m
import talib
import pickle

from hmmlearn.hmm import GaussianHMM
from sklearn.externals import joblib

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearRegression from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVR from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVC from version 0.18 when using versi

In [3]:
filename_test = '/home/octo/Dropbox'+ '/SPY21Dec.csv'

In [4]:
data=processing.get_csv_pd(filename_test)
data=processing.BA(data)
data=processing.preprocessing_mar(data)
data_km=ml.kalman_ma(data)
data['ckm']=data.Close-data_km
data['km']=data_km
data['rsi']=talib.RSI(np.array(data.Close.astype('float64')))
data['atr']=talib.ATR(np.array(data.High.astype('float64')),np.array(data.Low.astype('float64')),np.array(data.Close.astype('float64')),timeperiod=20)
data['mom']=talib.MOM(np.array(data.Close.astype('float64')), timeperiod=5)
data=data.dropna()
data['ret']=np.log(data.Close/data.Close.shift(300))
data['AB']=np.where(np.log(data.Close/data.Close.shift(300))>0,1,-1)
data=data[17000:25000].dropna()

In [5]:
data=data.dropna()

# outlier

L3p1_UD_prediction_model_saving_draft

In [6]:
filename_outlr = 'outlr.sav'
outlr_loaded = pickle.load(open(filename_outlr, 'rb'))
predictions_outlr = outlr_loaded.predict(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','ckm', 'rsi', 'atr','mom']])
data['outlr']=predictions_outlr
hmm_loaded = joblib.load("markov_hmm.pkl")  
predictions_hmm = hmm_loaded.predict(data[['Volume','spread','ckm', 'rsi','UD']])
data['h_state']=predictions_hmm

In [7]:
data['NAP']=np.where(data.outlr*np.argmin(np.diag(hmm_loaded.transmat_))<1,-1,1)

# UD

#### L3P1_strategy_research_Mar20_saving

In [42]:
data['UD']=np.where(data.Close>data.km*1.00001,1,np.where(data.Close<data.km*0.99999,-1,0))

In [43]:
data['J']=np.where(np.logical_and(data.Close.diff(300)>0.04,data.UD==1),1,
                     np.where(np.logical_and(data.Close.diff(300)<-0.04,data.UD==-1),-1,0))

#### RF
##### loading and prediction

### After oversampling
https://beckernick.github.io/oversampling-modeling/

https://elitedatascience.com/imbalanced-classes

In [44]:
filename_rf = 'rf.sav'
rf_loaded = pickle.load(open(filename_rf, 'rb'))
predictions_rf = rf_loaded.predict(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','rsi', 'atr', 'mom']])

In [45]:
rf_loaded.score(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','rsi', 'atr', 'mom']],data.J)

0.818875

In [46]:
data['prJ']=predictions_rf

In [47]:
# class distribution
print(data.groupby('prJ').size())

prJ
-1      43
 0    7014
 1     943
dtype: int64


In [14]:
# class distribution
print(data.groupby('NAP').size())

NAP
-1    2160
 1    5840
dtype: int64


In [15]:
pd.crosstab(index=data.NAP,columns=data.prJ) 

prJ,-1,0,1
NAP,,,
-1,17,2081,62
1,26,4933,881


## Sell and cover points

In [48]:
data['sc']=np.where(data.Close.diff(30)>0.05,1,np.where(data.Close.diff(30)<-0.05,-1,0))

In [49]:
filename_rf = 'rf-sc.sav'
rf_loaded = pickle.load(open(filename_rf, 'rb'))
predictions_sc = rf_loaded.predict(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','km','rsi', 'atr', 'mom']])

In [50]:
rf_loaded.score(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','km','rsi', 'atr', 'mom']],data.sc)

0.981375

In [39]:
data['prSC']=predictions_sc

In [40]:
# class distribution
print(data.groupby('prSC').size())

prSC
-1      61
 0    7851
 1      88
dtype: int64


In [41]:
pd.crosstab(index=data.NAP,columns=data.prSC) 

prSC,-1,0,1
NAP,,,
-1,31,2122,7
1,30,5729,81
